# Aufgabe 4 - Gaussian Mixture Models

In dieser Aufgabe wollen wir Flächen auf einem Satellitenbild nach Verwendung (land use) klassifizieren. Wir werden versuchen, Pixel mit ähnlichen Farben zu einer Klasse zu gruppieren und so unterschiedliche Landflächen unterscheiden können (Wald, Wüste, Stadt, etc.)

Zuerst laden wir das Bild:

Für diese Aufgabe benötigen wir die `pillow` Bibliothek. Falls die nächste Zelle bei ihnen einen Fehler produziert, installieren Sie diese mit `conda install pillow` oder `pip install pillow`.

In [ ]:
%matplotlib notebook
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

image = Image.open("data.png")
data = np.array(image)[:, :, :3].astype(float) / 255
print(data.shape)
image

Das Bild ist 500x500 pixel groß und hat drei Farbkanäle. Diese werden unsere features sein.

## Gaussian Mixture Models (GMM)

Ein GMM besteht aus $K$ Tupeln $(\alpha_k, \mu_k, \Sigma_k), i\in[k..K]$ für ein vorher festgelegtes $K$. Das GMM beschreibt eine Wahrscheinlichkeitsverteilung:

$$P(x)=\sum_{k=1}^K \alpha_k\mathcal{N}(\mu_k, \Sigma_k)(x)$$

Hier ist $\mathcal{N}$ die multivariate Gaußverteilung:

$$\mathcal{N}(\mu, \Sigma)(x)=\frac{1}{\sqrt{(2\pi)^d|\Sigma|}}\exp(-\frac{1}{2}(x-\mu)^\top\Sigma^{-1}(x-\mu))$$

$|\Sigma|$ ist die Determinante und $\Sigma^-1$ die Inverse von $\Sigma$.

Die $\alpha_k$ sind Verteilungsgewichte, (die a-priori Wahrscheinlichkeiten aus Bayes) und es gilt $\sum_{k=1}^K\alpha_k=1$.

Wir wollen GMMs zur Klassifikation benutzen, indem wir annehmen, dass jedes $k$ eine Klasse $\omega_k$ darstellt. Wir können jetzt für einen Datenpunkt $x_i$ ein Klassengewicht bestimmen:

$$w_{ik}=p(\omega_k|x_i)=\frac{P(x_i|\omega_k)p(\omega_k)}{P(x_i)}=\frac{P(x_i|\omega_k)p(\omega_k)}{\sum_{m=1}^K P(x_i|\omega_m)\alpha_m}=\frac{\mathcal{N}(\mu_k, \Sigma_k)(x_i)\alpha_k}{\sum_{m=1}^K \mathcal{N}(\mu_m, \Sigma_m)(x_i)\alpha_m}$$

Bei der Klassifikation wählen wir für $x_i$ die Klasse $\omega_k$ mit dem höchsten $w_{ik}$. Aber wir benötigen diese Gewichte auch beim Training der Parameter.

### Der EM-Algorithmus

Wir trainieren die Parameter mit dem Expectation-Maximization (EM) Algorithmus. Er besteht aus zwei Schritten, die abwechselnd wiederholt werden:

#### E-Schritt

Mit den aktuellen Parametern, berechne $w_{ik}$ für alle $x_i$ und alle $\omega_k$. Aufgrund der Definition der $w_{ik}$ gilt $\sum_{k=1}^K w_{ik}=1$. Das Ergebnis ist eine $N\times K$ Matrix in der sich jede Zeile zu 1 summiert. ($N$ sei die Anzahl der Datenpunkte).

#### M-Schritt

Benutze die Gewichte und die Daten um neue, bessere Schätzungen der Parameter zu bestimmen. Sei $N_k=\sum_{i=1}^N w_{ik}$. Die neuen Parameterwerte sind:

$$\alpha_k^{\text{neu}}=\frac{N_k}{N}$$

$$\mu_k^{\text{neu}}=\frac{1}{N_k}\sum_{i=1}^N w_{ik}\cdot x_i$$

$$\Sigma_k^{\text{neu}}=\frac{1}{N_k}\sum_{i=1}^N w_{ik}\cdot(x_i-\mu_k^{\text{neu}})(x_i-\mu_k^{\text{neu}})^\top$$

#### Initialisierung und Ende

Wir initialisieren die Parameter mit

$$\alpha_k=\frac{1}{K}$$

$$\mu_k=x_j\quad \text{j zufällig aus 1..N}$$

$$\Sigma_k=\frac{1}{N}\sum_{i=1}^N (x_i-\hat{\mu})(x_i-\hat{\mu})^\top$$

Hier ist $\hat{\mu}=\frac{1}{N}\sum_{i=1}^N x_i$. Das bedeutet, dass $\Sigma_k$ und $\alpha_k$ gleich für alle $k$ ist, aber $\mu_k$ nicht.

Um zu entscheiden, wann der Algorithmus beendet ist, berechnen wir nach jedem M-Schritt die log-likelihood:

$$\log l(X)=\sum_{i=1}^N \log P(x_i) = \sum_{i=1}^N \log \sum_{k=1}^K \alpha_k P(x_i|\omega_k)$$

Wenn die log-likelihood um weniger als 1% steigt, beenden wir den Algorithmus.

### Aufgabe a) EM-Algorithmus (30 Punkte)

Implementieren Sie den E-Schritt, den M-Schritt und die Berechnung der log-likelihood. Verändern Sie die vorgegebenen Methoden nicht, aber sie dürfen zusätzliche Hilfsmethoden schreiben.

Hinweise: Verwedenden Sie `np.linalg.inv` und `np.linalg.det` jeweils für das Inverse und die Determinante einer Matrix

In [ ]:
class GMM:
    def __init__(self, num_classes, num_dims):
        self.num_classes = num_classes
        self.num_dims = num_dims
        self.priors = np.empty(num_classes)
        self.means = np.empty((num_classes, num_dims))
        self.covariances = np.empty((num_classes, num_dims, num_dims))
    
    def fit(self, data):
        original_shape = data.shape
        assert data.shape[-1] == self.num_dims
        data = data.reshape(-1, self.num_dims)
        
        self.initialize_parameters(data)
        
        log_likelihood = self.log_likelihood(data) / len(data)
        
        while True:
            weights = self.e_step(data)
            assert weights.shape == (len(data), self.num_classes)
            assignments = np.argmax(weights, axis=1).reshape(original_shape[:-1])
            yield log_likelihood, assignments
            assert weights.shape == (len(data), self.num_classes)
            self.m_step(data, weights)
            new_likelihood = self.log_likelihood(data) / len(data)
            if new_likelihood < log_likelihood + 1e-3:
                break
            log_likelihood = new_likelihood
        yield new_likelihood, self.classify(data.reshape(original_shape))
    
    def classify(self, data):
        assert data.shape[-1] == self.num_dims
        flat_data = data.reshape(-1, self.num_dims)
        
        weights = self.e_step(flat_data)
        assignments = np.argmax(weights, axis=1)
        assignments = assignments.reshape(data.shape[:-1])
        return assignments
    
    def initialize_parameters(self, data):
        self.priors[:] = 1 / self.num_classes
        self.means[:] = data[np.random.randint(len(data), size=self.num_classes)]
        global_mean = data.mean(axis=0)
        diff = data - global_mean
        global_covariance = np.einsum("Ni,Nj->Nij", diff, diff).mean(axis=0)
        self.covariances[:] = global_covariance
    
    def log_likelihood(self, data):
        # Your code here
        pass
    
    def e_step(self, data):
        # Your code here
        pass
    
    def m_step(self, data, weights):
        # Your code here
        pass

Jetzt wenden wir den Algorithmus an. Sie können auch eine andere Anzahl Klassen wählen und den Effekt auf das Ergebnis beobachten.

Falls Ihre Implementierung Schwierigkeiten hat, die gesamten Daten zu verarbeiten, können Sie auch mit einem subset der Daten (jeder zweite, dritte, etc.) trainieren.
Für die volle Punktzahl sollte Ihre Implementierung mit mindestens 50x50 Datenpunkten in unter 2 Minuten trainieren können.

In [ ]:
gmm = GMM(num_classes=5, num_dims=3)
train_data = data
# train_data = data[::2,::2]
# train_data = data[::5,::5]
# train_data = data[::10,::10]

print(f"Training mit {train_data.size // 3} Datenpunkten")
for i, (likelihood, assignment) in enumerate(gmm.fit(data)):
    print(f"Iteration {i} | log-likelihood {likelihood:.3f}")

### Aufgabe b) Visualisiserung (10 Punkte)

Entwickeln Sie mindestens zwei unterschiedliche sinnvolle Visualisierungen für das Ergebnis Ihres Klassifikators.

Vorschläge:

* Mit `PIL.Image.fromarray` können Sie ein numpy array (Datentyp `np.uint8`, normalisiert auf `[0,255]`) in ein Bild umwandeln. Wenn die Ausgabe einer Zelle (also der Wert auf der letzten Zeile) das Bild ist, wird es im Notebook dargestellt
* Mit `matplotlib.pyplot.imshow` können Sie sowohl Bilder als auch heatmaps darstellen
* Mit `IPython.display.Markdown` können Sie in Python erzeugten Markdown code im Notebook darstellen lassen und damit z.B. eine Tabelle erzeugen

In [ ]:
# Dies sind nur Vorschläge für Größen, die Sie darstellen könnten:
assignments = gmm.classify(data)
means = gmm.means
covariances = gmm.covariances
class_counts = np.bincount(assignments.reshape(-1))

# Your code here